In [1]:
import binascii
import hashlib
import math
import struct

import bencodepy
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes


def uint32(data, offset):
    """Reads a 32-bit unsigned integer (little-endian) from the data."""
    return struct.unpack_from("<I", data, offset)[0]

class HAXFile:
    def __init__(self, file_data, keys):
        # Validate header length
        expected_header_length = uint32(file_data, 8)
        if len(file_data) < expected_header_length:
            raise ValueError("Incomplete HAX header")

        # File properties
        self.file_length = uint32(file_data, 4)
        self.header_length = expected_header_length
        self.extra_length = uint32(file_data, 12)

        # Decode metadata (e.g., using Bencode)
        metadata_raw = file_data[16:self.header_length]
        self.meta = bencodepy.decode(metadata_raw)

        # Base key
        self.base_key = self.meta[b"baseKey"]  # Raw base key

        # Parse segments
        self.segments = []
        for i in range(self.meta[b"segmentCount"]):
            offset = uint32(self.meta[b"segments"], i * 8)
            pts = uint32(self.meta[b"segments"], i * 8 + 4)
            self.segments.append({"off": offset, "pts": pts})

        # Add the last segment (end of the file)
        self.segments.append({
            "off": self.file_length,
            "pts": self.meta[b"durationMs"]
        })

        # Load decryption keys - convert hex strings to bytes
        self.keys = {int(k): binascii.unhexlify(v) for k, v in keys.items()}

    def segment_bounds(self, segment_index):
        """Returns the byte range for a given segment."""
        if segment_index < 0 or segment_index >= len(self.segments) - 1:
            return None
        return (self.segments[segment_index]["off"], self.segments[segment_index + 1]["off"])

    def derive_key(self, index):
        """Derives a cryptographic key for a given index."""
        r = math.ceil(math.log2(index)) if index > 0 else 0
        derived_key = None
        found_i = 0

        # Find the first applicable key by checking each bit position
        for i in range(r + 1):
            key_index = index >> (r - i)  # Shift right to get the key index
            if key_index in self.keys:
                derived_key = self.keys[key_index]
                found_i = i
                break

        if derived_key is None:
            # If no key is found, try using the base key
            if 1 in self.keys:  # Often the base key is stored with index 1
                derived_key = self.keys[1]
                found_i = 0
            else:
                raise ValueError("No applicable key available")

        # Iteratively derive the key using the remaining bits
        for i in range(found_i + 1, r + 1):
            shift_byte = bytes([index >> (r - i) & 0xFF])
            derived_key = hashlib.sha256(derived_key + shift_byte).digest()

        return derived_key

    def decode(self, segment_index, raw_data):
        """Decrypts a segment using its index and raw data."""
        # Compute h as in the JavaScript code
        h = 1 + (1 << (math.ceil(math.log2(len(self.segments))) + 1))

        # Derive the decryption key
        derived_key = self.derive_key(h + segment_index)

        # Pad the data to a multiple of 16 bytes if necessary
        padding_length = (16 - len(raw_data) % 16) % 16
        padded_data = raw_data + bytes([padding_length] * padding_length)

        # Initialize decryption with AES in CBC mode with zero IV
        iv = bytes(16)  # 16 bytes of zeros
        cipher = Cipher(algorithms.AES(derived_key), modes.CBC(iv), backend=default_backend())
        decryptor = cipher.decryptor()

        # Decrypt the data
        decrypted_data = decryptor.update(padded_data) + decryptor.finalize()

        # Remove padding
        if padding_length > 0:
            decrypted_data = decrypted_data[:-padding_length]

        return decrypted_data

In [11]:
listen_dict = {
    "url": "https://cdn.hotaudio.net/a/hm6aq9rrzwtt2drebe64hmjf00.hax",
    "length15s": 245797,
    "keys": {
        "32": "2c90525bd6fdc786146e0d239ee1fa73fe19d34b14e251adbd17ed3459b39c91",
        "33": "c1e3916a2fd7ed4912699e7a9055553233d6218ace9ee24b3b3947ee66e2aed1",
        "34": "1c3d68352776b05c29575f3871d2993c45b129c0c83a9a1ff76b0b00f6dca78d",
        "35": "eb35eab8bdf04d53ecaed302fd533c5d95494f24f004a73dbe1bf8b46f0a90c6",
        "36": "7d131390c0048c51608628d1d2fbe7faf93acb28f0c85d0017b82c4f23a44d00",
        "37": "a0e3c6e98423229cf95f73cbf82a48d8bb5533917a39723801d716e6de334559",
        "38": "a8bd5c9544ef675f0948b3e1ed6475375d107d645cf4855ad6792d8c6e1856b0",
        "39": "c08359e3b435a53813d1d622f47620288b8e310cbbd83c39d4e5738109cee7e8",
        "40": "3fa3b14d7ffec9341ac34b54fb136d83bf68bf49060802cc74c53b835aec6a12",
        "41": "8bf9097b1f748fb5413e8185b47d2aa27383ebe39ad746e9b50588fc656b0f04",
        "42": "5aaf87dfd6c3a57ab856acbbed8f95f314739b61fe193aa549ae730bc8958304",
        "43": "1f5f1869a4d260cafab5d497d3ae940e0c09dda4a9493bf0e6cda8cbdf9de19a",
        "44": "d6c00ead34bda427964d19cfa31a334b81c1deed7904682ba891eb8a90621097",
        "45": "e6447e63434987ffbcc9ab89e8408f5aaeaf8f09c59884a5191a24cfe7db43cd",
        "23": "4ed88aedba481a7b496b5c360318e1da0afec20627ccea5e536e17f8b1bdb275"
    }
}

In [ ]:
from pathlib import Path

with Path("hm6aq9rrzwtt2drebe64hmjf00.hax").open("rb") as f:
    file_data = f.read()

hax_file = HAXFile(file_data, listen_dict["keys"])

# Check segment bounds
for i in range(len(hax_file.segments) - 1):
    print(f"Segment {i}: {hax_file.segment_bounds(i)}")


In [ ]:
with Path("hm6aq9rrzwtt2drebe64hmjf00.hax").open("rb") as f:
    file_data = f.read()

hax_file = HAXFile(file_data, listen_dict["keys"])

# Decode a specific segment (replace 0 with the actual segment index)
segment_bounds = hax_file.segment_bounds(0)
raw_data = file_data[segment_bounds[0]:segment_bounds[1]]
decrypted_data = hax_file.decode(0, raw_data)

print(decrypted_data)


In [ ]:
with Path("hm6aq9rrzwtt2drebe64hmjf00.hax").open("rb") as f:
    file_data = f.read()

hax_file = HAXFile(file_data, listen_dict["keys"])

# Create output AAC file
with Path("output.aac").open("wb") as out_f:
    # Decode all segments
    for i in range(len(hax_file.segments) - 1):  # -1 because last segment is just end marker
        segment_bounds = hax_file.segment_bounds(i)
        if segment_bounds:
            raw_data = file_data[segment_bounds[0]:segment_bounds[1]]
            try:
                decrypted_data = hax_file.decode(i, raw_data)
                out_f.write(decrypted_data)
                print(f"Decoded segment {i}/{len(hax_file.segments)-2}")
            except Exception as e:
                print(f"Error decoding segment {i}: {e}")
                continue

print("Decoding complete. Output saved to output.aac")